# 🧠 LLM Deep Research Comparator (ChatGPT + Gemini)
A notebook to compare answers from GPT-4 and Gemini models for deep research tasks.

## 📦 Imports and Setup
This cell imports necessary libraries and loads your API keys from a `.env` file.

In [ ]:
import openai
import requests
import pandas as pd
from IPython.display import display, Markdown
import tiktoken
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_KEY")
GEMINI_KEY = os.getenv("GEMINI_KEY")

## 🔧 Model Configuration
Specify which OpenAI and Gemini models you want to use.

In [ ]:
OPENAI_MODELS = ["gpt-4o", "gpt-4-mini-high"]
GEMINI_MODEL = "gemini-pro"  # You can also try: "gemini-1.5-pro-latest" if supported


## 📝 Initial Research Prompt
Enter your initial research question or task here.

In [ ]:
initial_prompt = """
i am a senior economist at the GLA, being sent on this course (https://bse.eu/summer-school/crei-macroeconomics/quantitative-methods-spatial-economics) to improve my quantitative spatial modelling skills... [truncated for brevity]
"""

## 🔢 Token Counter
Helper function to estimate token count (OpenAI only).

In [ ]:
def count_tokens(text, model="gpt-4o"):
    try:
        enc = tiktoken.encoding_for_model(model)
    except:
        enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))

## 🔌 OpenAI Query Function
Sends prompt to OpenAI (GPT-4, GPT-4-mini) using the new v1+ client interface.

In [ ]:
def query_openai(prompt, model, key):
    try:
        client = openai.OpenAI(api_key=key)
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        content = response.choices[0].message.content
        usage = response.usage.total_tokens if hasattr(response, 'usage') else count_tokens(prompt, model)
        cost = usage * (0.00001 if model == "gpt-4-mini-high" else 0.00005)
        return content, usage, cost
    except Exception as e:
        return f"OpenAI Error: {str(e)}", 0, 0

## 🔌 Gemini Query Function
Sends prompt to Gemini using the public API endpoint.

In [ ]:
def query_gemini(prompt, key, model="gemini-pro"):
    try:
        url = f"https://generativelanguage.googleapis.com/v1beta/models/{model}:generateContent?key={key}"
        headers = {"Content-Type": "application/json"}
        data = {"contents": [{"parts": [{"text": prompt}]}]}
        response = requests.post(url, headers=headers, json=data)
        content = response.json()['candidates'][0]['content']['parts'][0]['text']
        tokens = count_tokens(prompt, "gpt-4o")  # Approximate
        return content, tokens, 0
    except Exception as e:
        return f"Gemini Error: {str(e)}", 0, 0

## 🚀 Run Prompt Across Models
This function submits the prompt to all configured models and collects results.

In [ ]:
def run_round(prompt_text, label="Initial"):
    results = []

    for model in OPENAI_MODELS:
        answer, tokens, cost = query_openai(prompt_text, model, OPENAI_KEY)
        results.append({"model": model, "provider": "OpenAI", "response": answer, "tokens": tokens, "cost": cost, "label": label})

    gemini_ans, tokens, cost = query_gemini(prompt_text, GEMINI_KEY, model=GEMINI_MODEL)
    results.append({"model": GEMINI_MODEL, "provider": "Google", "response": gemini_ans, "tokens": tokens, "cost": cost, "label": label})

    return pd.DataFrame(results)

## 📥 Run Initial Prompt
Runs your prompt and shows all model responses side by side.

In [ ]:
all_responses = run_round(initial_prompt, label="Initial")

for _, row in all_responses.iterrows():
    display(Markdown(f"## 🤖 {row['provider']} – {row['model']} ({row['label']})"))
    display(Markdown(f"**Tokens:** {row['tokens']} | **Est. Cost ($):** {row['cost']:.5f}"))
    display(Markdown(row['response'][:10000]))

## 🔄 Optional Follow-Up Prompt
Allows you to enter a follow-up and rerun across all models.

In [ ]:
follow_up = input("\nType a follow-up prompt or press Enter to skip: ")
if follow_up.strip():
    follow_responses = run_round(follow_up, label="Follow-up")
    all_responses = pd.concat([all_responses, follow_responses], ignore_index=True)
    for _, row in follow_responses.iterrows():
        display(Markdown(f"## 🔁 {row['provider']} – {row['model']} ({row['label']})"))
        display(Markdown(f"**Tokens:** {row['tokens']} | **Est. Cost ($):** {row['cost']:.5f}"))
        display(Markdown(row['response'][:10000]))

## 📝 Export Results to Markdown
Saves everything to a Markdown file you can import into Notion or Obsidian.

In [ ]:
md = ""
for _, row in all_responses.iterrows():
    md += f"\n\n## 🤖 {row['provider']} – {row['model']} ({row['label']})\n"
    md += f"**Tokens:** {row['tokens']} | **Est. Cost ($):** {row['cost']:.5f}\n\n"
    md += row['response']

with open("llm_comparison.md", "w", encoding="utf-8") as f:
    f.write(md)

print("\n✅ All responses saved to llm_comparison.md with token and cost tracking.")